##### This jupyter notebooks is analyze which transcription factors or miRNAs that regulate the expression of genes using a Lasso regression model
##### Date: Feb 22, 2023

In [2]:
#import libraries
import pandas as pd
from sklearn import linear_model 
import numpy as np

In [4]:
GE = pd.read_csv("../../../Data_resource/RPKM.csv",index_col = "Unnamed: 0")
GE.index = GE['Gene']

In [5]:
#gene id mapping
gene_map = GE.loc[:,['Gene','Symbol']]
dic_gene_map = {}
for i in range(0,gene_map.shape[0]):
    dic_gene_map[gene_map.iloc[i,0]] = gene_map.iloc[i,1]
    dic_gene_map[gene_map.iloc[i,1]] = gene_map.iloc[i,0]
    
# full gene list and sample list
gene_list = (list(GE.index))
sample_list = list(set(GE.columns) - set(['Gene','Symbol']))

# Formated input gene expression matrix
input_data = GE.loc[list(gene_list), sample_list]

In [6]:
def Filter_low_Expr(arr):
    '''
    Usage: This function is uded to filter out genes which has expression value logRPKM smaller than 0 in more than 50% of the samples.
    Parameter: arr, expression of one gene across different samples
    '''
    total_len = len(arr)
    count = 0
    for i in arr:
        if i > 0:
            count = count + 1
    if count > 0.5*total_len: #If 
        return(True)
    else:
        return(False)

In [7]:
# Filter genes and scale the data

scaled_df = pd.DataFrame()
array_list = list()
length = input_data.shape[0]
gene_list_remain = []
for i in range(0,length):
    gene = gene_list[i]
    arr = input_data.iloc[i,:]
    if Filter_low_Expr(arr):
        array_mean = np.mean(arr)
        array_std = np.std(arr)
        array_scale = (arr - array_mean)/array_std
        array_list.append(array_scale.values)
        gene_list_remain.append(gene)
#cur_df = pd.DataFrame({gene: array_scale})
#scaled_df = pd.concat([scaled_df, cur_df], axis =1 )
    
scaled_df = pd.DataFrame(array_list)
scaled_df.columns = sample_list
scaled_df.index = gene_list_remain

In [8]:
# Load the transcription factor and target gene pairs
TFs = pd.read_csv("~/Documents/GitHub_project/fastqpi_BigGIM/KGs/TFs/database.csv") # source from doi:10.1101/gr.240663.118 (2019)

In [9]:
TFs_sele = TFs.loc[TFs['score'].isin(['A','B','C','D','E'])]

In [3]:
miRNA_tar_KG = pd.read_csv("./Data/KG_miRNA_target_edges.csv") # source from miRTarBase and MSigDB
#miRNA_list = []
#for i in set(GE['Symbol']):
#    if 'MIR' in i:
#        miRNA_list.append(i)
        

In [26]:
miRNA_list = list(set(miRNA_tar_KG['Subject']).intersection(set(GE['Symbol'])))

In [27]:
#miRNA_tar_KG.loc[miRNA_tar_KG['Subject'].isin(set(GE['Symbol']).intersection(set(miRNA_tar_KG['Subject'])))]

In [28]:
miRNA_tar_KG_sub = miRNA_tar_KG.loc[miRNA_tar_KG['Subject'].isin(miRNA_list)]

In [76]:
#Potential_regulator_list

### select regulators for one target gene

In [84]:
result_list = []
count = 0
new_coef = []

coef_list_all = []
target_list = []
regulator_list = []
r2_list_all = []

for sele_gene in gene_list_remain:
    
#for sele_gene in [dic_gene_map['BCL2']]:
    print(sele_gene)
    new_coef = []
    count = count + 1
    if count % 1000 ==0:
        print(count)
    
    #if sele_gene in set(TFs_sele['target']):
    if sele_gene in dic_gene_map:
        sele_TF_curr_gene = list(TFs_sele.loc[TFs_sele['target'] == dic_gene_map[sele_gene]]['TF'])
    
        if sele_gene in sele_TF_curr_gene:
            sele_TF_curr_gene.remove(sele_gene)
    else:
        sele_TF_curr_gene = []
        
    sele_miRNA_curr_gene = list(miRNA_tar_KG_sub.loc[miRNA_tar_KG_sub['Object'] == sele_gene]['Subject'])

    Potential_regulator_list = sele_TF_curr_gene + miRNA_list
    print(len(Potential_regulator_list))
    
    X_regulators_symbol = []
    X_regulators = []
    for i in Potential_regulator_list:
        if i in dic_gene_map:
            if dic_gene_map[i] in  scaled_df.index:
                X_regulators.append(dic_gene_map[i])
                X_regulators_symbol.append(i)

    #print(len(X_regulators_symbol))
    #Y_gene =dic_gene_map[sele_gene]
    Y_gene = sele_gene
    Y = scaled_df.loc[Y_gene,:].values
    X = scaled_df.loc[X_regulators,:]
    X = X.transpose()


    linreg = linear_model.Lasso(alpha=0.1) 
    linreg.fit(X,Y)
    y_pred = linreg.predict(X)

    coef = list(linreg.coef_)

    new_coef.extend(coef)
    rss = sum((y_pred-Y)**2)
    new_coef.extend([rss])
    new_coef.extend([linreg.intercept_])
    new_coef.extend([r2_score(Y, y_pred)])
    
    print("R2:" +str(r2_score(Y, y_pred)))
    #print(r2_score(Y, y_pred))
    print('nonzero coefs: ', sum(linreg.coef_ != 0))

   
    print("MES:")
    print(np.mean((y_pred - Y)**2))
    
    
    
    gene_cur = X_regulators_symbol
    gene_cur.append(sele_gene+"_"+"RSS")
    gene_cur.append(sele_gene+"_"+"intercept_") 
    gene_cur.append(sele_gene+"_"+"R2") 
    
    result_cur = pd.DataFrame({"Regulator":X_regulators_symbol,
                                "coef":new_coef,
                               "target_gene":[sele_gene]*len(new_coef)})
    result_cur = result_cur.loc[result_cur['coef']!=0]
    regulator_list.extend(result_cur['Regulator'].values)
    target_list.extend(result_cur['target_gene'].values)
    coef_list_all.extend( result_cur['coef'].values)

ENSG00000000419
85
R2:0.2901330876038154
nonzero coefs:  9
MES:
0.7098669123961846
ENSG00000000457
157
R2:0.5699401368350462
nonzero coefs:  23
MES:
0.4300598631649537
ENSG00000000460
132
R2:0.5348766283619488
nonzero coefs:  11
MES:
0.46512337163805106
ENSG00000000938
145
R2:0.7955816283268701
nonzero coefs:  17
MES:
0.2044183716731299
ENSG00000000971
99
R2:0.2194533501663184
nonzero coefs:  11
MES:
0.7805466498336814
ENSG00000001036
93
R2:0.4995201762685181
nonzero coefs:  17
MES:
0.5004798237314819
ENSG00000001084
127
R2:0.34842597807724196
nonzero coefs:  13
MES:
0.6515740219227583
ENSG00000001167
120
R2:0.99
nonzero coefs:  1
MES:
0.01000000000000004
ENSG00000001460
125
R2:0.638854068976958
nonzero coefs:  17
MES:
0.36114593102304204
ENSG00000001461
100
R2:0.6439237390307266
nonzero coefs:  14
MES:
0.35607626096927325
ENSG00000001497
115
R2:0.6664982157245879
nonzero coefs:  11
MES:
0.3335017842754121
ENSG00000001561
80
R2:0.24217349049597137
nonzero coefs:  11
MES:
0.757826509504

R2:0.553252962632894
nonzero coefs:  22
MES:
0.44674703736710597
ENSG00000006282
104
R2:0.2622700755103783
nonzero coefs:  11
MES:
0.7377299244896216
ENSG00000006327
101
R2:0.6582774915023717
nonzero coefs:  13
MES:
0.3417225084976283
ENSG00000006451
109
R2:0.4729443663701379
nonzero coefs:  17
MES:
0.5270556336298621
ENSG00000006459
136
R2:0.6642071057014104
nonzero coefs:  13
MES:
0.33579289429858966
ENSG00000006530
110
R2:0.22499771553437187
nonzero coefs:  18
MES:
0.7750022844656281
ENSG00000006534
109
R2:0.5997280348554884
nonzero coefs:  18
MES:
0.4002719651445117
ENSG00000006576
142
R2:0.5125892375012262
nonzero coefs:  20
MES:
0.48741076249877385
ENSG00000006607
106
R2:0.393031755926898
nonzero coefs:  17
MES:
0.606968244073102
ENSG00000006625
116
R2:0.620443757389
nonzero coefs:  12
MES:
0.379556242611
ENSG00000006634
109
R2:0.7899268293431209
nonzero coefs:  11
MES:
0.2100731706568792
ENSG00000006638
91
R2:0.3801926019026548
nonzero coefs:  10
MES:
0.6198073980973452
ENSG0000

ENSG00000010803
153
R2:0.6106619497034028
nonzero coefs:  10
MES:
0.3893380502965972
ENSG00000010810
258
R2:0.587221313339259
nonzero coefs:  19
MES:
0.4127786866607411
ENSG00000010818
218
R2:0.5290848736691958
nonzero coefs:  17
MES:
0.4709151263308042
ENSG00000011007
100
R2:0.5899060241076544
nonzero coefs:  20
MES:
0.41009397589234564
ENSG00000011009
139
R2:0.3743609511935112
nonzero coefs:  17
MES:
0.6256390488064889
ENSG00000011021
106
R2:0.4031866386223123
nonzero coefs:  12
MES:
0.5968133613776877
ENSG00000011028
153
R2:0.3850848936588923
nonzero coefs:  14
MES:
0.6149151063411075
ENSG00000011114
147
R2:0.40492285061724864
nonzero coefs:  16
MES:
0.5950771493827514
ENSG00000011132
146
R2:0.5111912397017209
nonzero coefs:  18
MES:
0.48880876029827913
ENSG00000011143
141
R2:0.7934082232514703
nonzero coefs:  19
MES:
0.20659177674852974
ENSG00000011198
103
R2:0.6749537334451114
nonzero coefs:  10
MES:
0.3250462665548886
ENSG00000011243
125
R2:0.61958218880254
nonzero coefs:  20
MES

124
R2:0.5472006029922846
nonzero coefs:  20
MES:
0.45279939700771543
ENSG00000020922
103
R2:0.5179520164270437
nonzero coefs:  20
MES:
0.48204798357295625
ENSG00000021300
110
R2:0.3479691420977711
nonzero coefs:  19
MES:
0.6520308579022288
ENSG00000021355
106
R2:0.27379513696529156
nonzero coefs:  17
MES:
0.7262048630347084
ENSG00000021574
125
R2:0.44220775969638193
nonzero coefs:  20
MES:
0.5577922403036178
ENSG00000021762
166
R2:0.4823718756949925
nonzero coefs:  19
MES:
0.5176281243050075
ENSG00000021776
139
R2:0.32547858524908124
nonzero coefs:  27
MES:
0.6745214147509188
ENSG00000022267
151
R2:0.3874022028902139
nonzero coefs:  22
MES:
0.6125977971097861
ENSG00000022277
106
R2:0.40488708176017674
nonzero coefs:  18
MES:
0.5951129182398232
ENSG00000022556
74
R2:0.04659249883464167
nonzero coefs:  7
MES:
0.9534075011653583
ENSG00000022567
94
R2:0.4280906706587979
nonzero coefs:  15
MES:
0.5719093293412022
ENSG00000022840
107
R2:0.4993510824802161
nonzero coefs:  17
MES:
0.500648917

121
R2:0.45092557229569585
nonzero coefs:  19
MES:
0.5490744277043041
ENSG00000034677
152
R2:0.7071106622783809
nonzero coefs:  12
MES:
0.29288933772161907
ENSG00000034693
89
R2:0.38362776818411637
nonzero coefs:  12
MES:
0.6163722318158837
ENSG00000034713
87
R2:0.3142173057963471
nonzero coefs:  11
MES:
0.685782694203653
ENSG00000035115
183
R2:0.5739106558805416
nonzero coefs:  21
MES:
0.42608934411945854
ENSG00000035141
86
R2:0.636318097691877
nonzero coefs:  13
MES:
0.36368190230812303
ENSG00000035403
159
R2:0.398449921934818
nonzero coefs:  18
MES:
0.601550078065182
ENSG00000035499
98
R2:0.7935640086249282
nonzero coefs:  5
MES:
0.2064359913750719
ENSG00000035664
106
R2:0.527648507148083
nonzero coefs:  16
MES:
0.47235149285191697
ENSG00000035681
137
R2:0.6000309254045229
nonzero coefs:  19
MES:
0.399969074595477
ENSG00000035687
110
R2:0.5332783445556304
nonzero coefs:  17
MES:
0.46672165544436967
ENSG00000035720
82
R2:0.19031977795475707
nonzero coefs:  10
MES:
0.8096802220452427


...

50
R2:0.39036059631375863
nonzero coefs:  6
MES:
0.6096394036862411
ENSG00000270236
50
R2:0.42793955717150056
nonzero coefs:  9
MES:
0.5720604428284994
ENSG00000270248
50
R2:0.32482793136920973
nonzero coefs:  11
MES:
0.6751720686307903
ENSG00000270277
50
R2:0.17806975309073103
nonzero coefs:  9
MES:
0.8219302469092687
ENSG00000270294
50
R2:0.40593924603249976
nonzero coefs:  13
MES:
0.5940607539675002
ENSG00000270307
50
R2:0.3800974342669101
nonzero coefs:  5
MES:
0.6199025657330898
ENSG00000270344
50
R2:0.33312897355990234
nonzero coefs:  11
MES:
0.6668710264400977
ENSG00000270346
50
R2:0.2734112646342026
nonzero coefs:  14
MES:
0.7265887353657975
ENSG00000270362
50
R2:0.43923505764286863
nonzero coefs:  16
MES:
0.5607649423571313
ENSG00000270419
51
R2:0.36005952228767324
nonzero coefs:  13
MES:
0.6399404777123268
ENSG00000270426
50
R2:0.3638213803008177
nonzero coefs:  13
MES:
0.6361786196991823
ENSG00000270441
50
R2:0.4170156155556075
nonzero coefs:  12
MES:
0.5829843844443926
ENSG

R2:0.23220349220827352
nonzero coefs:  10
MES:
0.7677965077917265
ENSG00000271769
50
R2:0.5411833065195445
nonzero coefs:  10
MES:
0.45881669348045545
ENSG00000271780
50
R2:0.40206364394308214
nonzero coefs:  12
MES:
0.5979363560569176
ENSG00000271781
50
R2:0.12077631612026851
nonzero coefs:  9
MES:
0.8792236838797313
ENSG00000271788
50
R2:0.40457216930949813
nonzero coefs:  11
MES:
0.5954278306905019
ENSG00000271789
50
R2:0.6396981724043289
nonzero coefs:  12
MES:
0.36030182759567114
ENSG00000271795
50
R2:0.4494375908948052
nonzero coefs:  10
MES:
0.5505624091051948
ENSG00000271797
50
R2:0.3334616103782949
nonzero coefs:  11
MES:
0.6665383896217051
ENSG00000271816
50
R2:0.5929784748718296
nonzero coefs:  9
MES:
0.4070215251281703
ENSG00000271835
50
R2:0.44612539216403746
nonzero coefs:  14
MES:
0.5538746078359623
ENSG00000271840
50
R2:0.3078177024153539
nonzero coefs:  11
MES:
0.6921822975846461
ENSG00000271843
50
R2:0.3757563929119797
nonzero coefs:  13
MES:
0.6242436070880203
ENSG00

50
R2:0.24797198106923024
nonzero coefs:  13
MES:
0.7520280189307698
ENSG00000272221
50
R2:0.05830305963531213
nonzero coefs:  4
MES:
0.9416969403646879
ENSG00000272223
50
R2:0.3078335502059818
nonzero coefs:  11
MES:
0.6921664497940182
ENSG00000272247
50
R2:0.14115972994791892
nonzero coefs:  11
MES:
0.8588402700520813
ENSG00000272255
50
R2:0.4724762013746664
nonzero coefs:  12
MES:
0.5275237986253335
ENSG00000272256
50
R2:0.37343430552307433
nonzero coefs:  14
MES:
0.6265656944769257
ENSG00000272267
50
R2:0.29098782584604665
nonzero coefs:  13
MES:
0.7090121741539535
ENSG00000272269
50
R2:0.27946516634446605
nonzero coefs:  11
MES:
0.7205348336555342
ENSG00000272273
50
R2:0.4132851176531327
nonzero coefs:  13
MES:
0.5867148823468672
ENSG00000272275
50
R2:0.11964967535890292
nonzero coefs:  9
MES:
0.8803503246410971
ENSG00000272277
50
R2:0.38206406690531813
nonzero coefs:  13
MES:
0.6179359330946819
ENSG00000272279
50
R2:0.2588868886814435
nonzero coefs:  7
MES:
0.7411131113185567
ENS

ENSG00000272732
50
R2:0.32664070182830607
nonzero coefs:  11
MES:
0.6733592981716939
ENSG00000272733
50
R2:0.1856092773993827
nonzero coefs:  5
MES:
0.8143907226006174
ENSG00000272734
50
R2:0.1937040447381223
nonzero coefs:  6
MES:
0.8062959552618777
ENSG00000272746
50
R2:0.1393297570636124
nonzero coefs:  12
MES:
0.8606702429363876
ENSG00000272750
50
R2:0.3127067125255546
nonzero coefs:  19
MES:
0.6872932874744454
ENSG00000272754
50
R2:0.5796664742749333
nonzero coefs:  13
MES:
0.4203335257250666
ENSG00000272755
50
R2:0.04029028719701966
nonzero coefs:  4
MES:
0.9597097128029806
ENSG00000272758
50
R2:0.414762655641957
nonzero coefs:  10
MES:
0.5852373443580431
ENSG00000272760
50
R2:0.6002747836252973
nonzero coefs:  10
MES:
0.3997252163747027
ENSG00000272764
50
R2:0.39977320284752405
nonzero coefs:  9
MES:
0.6002267971524762
ENSG00000272767
50
R2:0.09055996606707684
nonzero coefs:  8
MES:
0.909440033932923
ENSG00000272768
50
R2:0.40635497289180245
nonzero coefs:  14
MES:
0.59364502710

50
R2:0.2444388827760695
nonzero coefs:  10
MES:
0.7555611172239306
ENSG00000273156
50
R2:0.24636101828560708
nonzero coefs:  9
MES:
0.7536389817143928
ENSG00000273162
50
R2:0.2806253502223761
nonzero coefs:  7
MES:
0.7193746497776239
ENSG00000273165
50
R2:0.5476703740805696
nonzero coefs:  11
MES:
0.4523296259194304
ENSG00000273174
50
R2:0.5309704914295758
nonzero coefs:  13
MES:
0.4690295085704243
ENSG00000273175
50
R2:0.3524515467362189
nonzero coefs:  17
MES:
0.6475484532637812
ENSG00000273183
50
R2:0.5669859372071016
nonzero coefs:  10
MES:
0.43301406279289845
ENSG00000273188
50
R2:0.39553744547289227
nonzero coefs:  11
MES:
0.6044625545271077
ENSG00000273190
50
R2:0.23074324796126877
nonzero coefs:  12
MES:
0.7692567520387312
ENSG00000273192
50
R2:0.4223152720071932
nonzero coefs:  8
MES:
0.5776847279928068
ENSG00000273196
50
R2:0.30834010753370344
nonzero coefs:  13
MES:
0.6916598924662966
ENSG00000273204
50
R2:0.23515754163088787
nonzero coefs:  12
MES:
0.7648424583691122
ENSG0

In [85]:
result_df = pd.DataFrame({"Target_gene":target_list, "Regulator":regulator_list, "Coef": coef_list_all})

In [86]:
result_df

,Target_gene,Regulator,Coef
0,ENSG00000000419,CDC5L,1.701583e-01
1,ENSG00000000419,GMEB2,2.731314e-02
2,ENSG00000000419,LYAR,1.412478e-02
3,ENSG00000000419,NFE2L2,8.039350e-02
4,ENSG00000000419,PHF5A,2.317665e-01
...,...,...,...
295131,ENSG00000273488,MIR3916,2.218090e-01
295132,ENSG00000273488,MIR4461,2.067389e-02
295133,ENSG00000273488,ENSG00000273488_RSS,2.695434e+02
295134,ENSG00000273488,ENSG00000273488_intercept_,3.388301e-16


In [246]:
#result_df.to_csv("KG_regulatory_graph_Mar1_2023.csv")

In [87]:
Symbol_list = []
for gene in list(result_df['Target_gene']):
    if gene in dic_gene_map:
        Symbol_list.append(dic_gene_map[gene])
    else:
        Symbol_list.append(gene)
        
result_df['Target_symbol'] = Symbol_list
result_df
#result_df.to_csv("KG_regulatory_graph.csv")

,Target_gene,Regulator,Coef,Target_symbol
0,ENSG00000000419,CDC5L,1.701583e-01,DPM1
1,ENSG00000000419,GMEB2,2.731314e-02,DPM1
2,ENSG00000000419,LYAR,1.412478e-02,DPM1
3,ENSG00000000419,NFE2L2,8.039350e-02,DPM1
4,ENSG00000000419,PHF5A,2.317665e-01,DPM1
...,...,...,...,...
295131,ENSG00000273488,MIR3916,2.218090e-01,RP11-114I8.4
295132,ENSG00000273488,MIR4461,2.067389e-02,RP11-114I8.4
295133,ENSG00000273488,ENSG00000273488_RSS,2.695434e+02,RP11-114I8.4
295134,ENSG00000273488,ENSG00000273488_intercept_,3.388301e-16,RP11-114I8.4


In [89]:
result_df.to_csv("KG_regulatory_graph_Mar1_2023.csv")
#This is related to the Supplementary table 2 in the paper.

# End